##### **Note**: 
> - Mọi API về product đều bị Shopee chặn
>
> - Shopee sử dụng Dynamic Web để load sản phẩm => Load hết page để lấy html => Selenium
>
> - Mình có thử 2 phương pháp để cào dữ liệu, kết quả đều bị shopee phát hiện có sử dụng tool auto hoặc bot
>
> - Phương pháp tối ưu nhất vẫn là hợp tác với bên Shopee Open API để tiết kiệm tài nguyên máy và cào được lượng lớn dữ liệu

## 1. Selenium + Selectolax

**requirement.txt**<br>
beautifulsoup4==4.12.3 <br>
urllib3==2.2.0 <br>
regex==2023.12.25 <br>
pandas==2.2.0 <br> 
lxml==5.1.0 <br>
requests==2.31.0<br>
html5lib==1.1<br>
Scrapy==2.11.0<br>
rich==13.7.0<br>
pydantic==2.6.0<br>
pydantic_core==2.16.1<br>
requests-html==0.10.0<br>
httpx==0.26.0<br>
selectolax==0.3.19<br>
tqdm==4.66.1<br>
undetected-chromedriver==3.5.4<br>

##### Import library

In [3]:
import urllib.request
from urllib.parse import quote
import pandas as pd
import warnings
import requests
from tqdm import tqdm
from selectolax.lexbor import LexborHTMLParser
from selenium import webdriver
import concurrent.futures
import undetected_chromedriver as uc
from pandas import json_normalize
warnings.filterwarnings("ignore")

C:\Users\duy6065\AppData\Local\Temp\ipykernel_19076\1101040555.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [103]:
def encode_string(input_string):
    encoded_string = quote(input_string, safe='')
    encoded_string = encoded_string.replace('%20', '-')
    return encoded_string

def text_percent_to_float(text):
    return float(text.replace("%", ""))/100
def convert_to_numeric(value_str):
    value_str = value_str.replace(',', '')  # Remove commas
    if 'k' in value_str:
        # Remove 'k' and convert to float, then multiply by 1000
        return int(float(value_str.replace('k', '')) * 100)
    else:
        return int(value_str)

def get_shopee_cat():
    headers = {
        'sec-ch-ua': '"Not A(Brand";v="99", "Microsoft Edge";v="121", "Chromium";v="121"',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0',
    }

    response = requests.get('https://shopee.vn/api/v4/pages/get_category_tree', headers=headers)
    

    df_1 = response.json()["data"]["category_list"]
    df = pd.DataFrame(df_1)
    df = df[['catid', 'display_name']]
    df_2 = pd.DataFrame()

    for i in range(len(df_1)):
        list_json = df_1[i]["children"]
        list_json = json_normalize(list_json)[["catid", "parent_catid", "display_name", "level"]]
        
        df_2 = pd.concat([df_2, list_json], ignore_index=True)
        df_2['urls'] = "https://shopee.vn/" + df_2['display_name'].str.replace(' &', '').str.replace(' ', '-') + '-cat.' + df_2['catid'].astype(str)
        
    df_3 = pd.merge(df, df_2, left_on='catid', right_on='parent_catid', how='left')
    df_3

    return df_3

##### Tạo df chứa url các nhóm hàng

In [105]:
df = get_shopee_cat()
df

,catid_x,display_name_x,catid_y,parent_catid,display_name_y,level,urls
0,11035567,Thời Trang Nam,11035568,11035567,Áo Khoác,2,https://shopee.vn/Áo-Khoác-cat.11035568
1,11035567,Thời Trang Nam,11035572,11035567,Áo Vest và Blazer,2,https://shopee.vn/Áo-Vest-và-Blazer-cat.11035572
2,11035567,Thời Trang Nam,11035578,11035567,"Áo Hoodie, Áo Len & Áo Nỉ",2,"https://shopee.vn/Áo-Hoodie,-Áo-Len-Áo-Nỉ-cat...."
3,11035567,Thời Trang Nam,11035583,11035567,Quần Jeans,2,https://shopee.vn/Quần-Jeans-cat.11035583
4,11035567,Thời Trang Nam,11035584,11035567,Quần Dài/Quần Âu,2,https://shopee.vn/Quần-Dài/Quần-Âu-cat.11035584
...,...,...,...,...,...,...,...
284,11116484,Dụng cụ và thiết bị tiện ích,11116487,11116484,Dụng cụ cầm tay,2,https://shopee.vn/Dụng-cụ-cầm-tay-cat.11116487
285,11116484,Dụng cụ và thiết bị tiện ích,11116485,11116484,Dụng cụ điện và thiết bị lớn,2,https://shopee.vn/Dụng-cụ-điện-và-thiết-bị-lớn...
286,11116484,Dụng cụ và thiết bị tiện ích,11116489,11116484,Thiết bị mạch điện,2,https://shopee.vn/Thiết-bị-mạch-điện-cat.11116489
287,11116484,Dụng cụ và thiết bị tiện ích,11116488,11116484,Vật liệu xây dựng,2,https://shopee.vn/Vật-liệu-xây-dựng-cat.11116488


In [ ]:
urls = df['urls'].tolist()

def get_html(url):
    options = webdriver.ChromeOptions()
    # driver = webdriver.Remote(command_executor="https://localhost:4444", options=options, )
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    html = driver.page_source
    driver.quit()
    return html

def parse_html(html):
    data = LexborHTMLParser(html)

    # products = {
    # "products": []
    # }

    product_name = data.css_first("div.efwNRW div.DgXDzJ.rolr6k.Zvjf4O").text()
    product_url = "https://shopee.vn" + data.css_first("[data-sqe='link']").attrs['href']

    product_rating = sum([text_percent_to_float(rate.attrs['style'].split(" ")[-1].replace(";", "")) for rate in data.css("div.shopee-rating-stars__star-wrapper div.shopee-rating-stars__lit")])
    
    product_price = float(data.css_first("div.bPcAVl.IWBsMB span.k9JZlv").text().replace(".", ""))
    product_revenue = convert_to_numeric(data.css_first("div.OwmBnn.eumuJJ").text().split(" ")[-1]) * product_price

    product = {
        'product_name': product_name,
        'product_url': product_url,
        'product_rating': round(product_rating, 2),
        'product_price': round(product_price),
        'product_revenue': round(product_revenue)
    }

    # products["products"].append(product)
    return product

    

with concurrent.futures.ThreadPoolExecutor(max_workers = 8) as executor:
    results = list(executor.map(get_html, urls))

for result in results:
    parse_html(result)

    

#####  Shopee phát hiện bot nên chuyển hướng sang web đăng nhập dù đã add cookie
![Đăng nhập](./img.png 'hehe')

---

## 2. Pyautogui + soup

##### 
1. Khả năng cao bị shopee request captcha
2. Tốc độ ~250 products/min, crawl max được (289 nhóm hàng, 9 page mỗi nhóm hàng, mỗi page 60 sản phẩm) ~ 160.000 products
=> Mất trên 10 tiếng

In [14]:
import webbrowser
import os
import time
from datetime import datetime
import pyautogui
pyautogui.FAILSAFE=False
import random
import pyperclip
import requests
import pandas as pd
from pandas import json_normalize
from datetime import datetime
from selectolax.lexbor import LexborHTMLParser
from rich import print

start = datetime.now()


def encode_string(input_string):
    encoded_string = quote(input_string, safe='')
    encoded_string = encoded_string.replace('%20', '-')
    return encoded_string

def text_percent_to_float(text):
    return float(text.replace("%", ""))/100
def convert_to_numeric(value_str):
    value_str = value_str.replace(',', '')  # Remove commas
    if 'k' in value_str:
        # Remove 'k' and convert to float, then multiply by 1000
        return int(float(value_str.replace('k', '')) * 100)
    else:
        return int(value_str)

def get_shopee_cat():
    headers = {
        'sec-ch-ua': '"Not A(Brand";v="99", "Microsoft Edge";v="121", "Chromium";v="121"',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0',
    }

    response = requests.get('https://shopee.vn/api/v4/pages/get_category_tree', headers=headers)
    

    df_1 = response.json()["data"]["category_list"]
    df = pd.DataFrame(df_1)
    df = df[['catid', 'display_name']]
    df_2 = pd.DataFrame()

    for i in range(len(df_1)):
        list_json = df_1[i]["children"]
        list_json = json_normalize(list_json)[["catid", "parent_catid", "display_name", "level"]]
        
        df_2 = pd.concat([df_2, list_json], ignore_index=True)
        df_2['urls'] = "https://shopee.vn/" + df_2['display_name'].str.replace(' &', '').str.replace(' ', '-') + '-cat.' + df_2['catid'].astype(str)
        
    df_3 = pd.merge(df, df_2, left_on='catid', right_on='parent_catid', how='left')
    df_3

    return df_3

def get_html(url):
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    script = '''
    copy(document.documentElement.outerHTML)
    '''
    webbrowser.open_new_tab(url)
    time.sleep(3)
    pyautogui.click(256,1056)
    for i in range(6):
        pyautogui.press('pagedown')
        time.sleep(1 + random.randint(0, 1))
    pyperclip.copy(script)
    pyautogui.hotkey('ctrl', 'shift', 'j')
    time.sleep(0.5)
    pyautogui.click(1800, 900)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(0.5)
    pyautogui.press('enter')
    time.sleep(0.5)
    pyautogui.hotkey('ctrl', 'w')
    html = pyperclip.paste()

    return html

def parse_html(html):
    datas = LexborHTMLParser(html)

    products = {
    "products": []
    }

    for data in datas.css("li.col-xs-2-4.shopee-search-item-result__item"):

        product_name = data.css_first("div.efwNRW div.DgXDzJ.rolr6k.Zvjf4O").text()
        product_url = "https://shopee.vn" + data.css_first("[data-sqe='link']").attrs['href']

        product_rating = sum([text_percent_to_float(rate.attrs['style'].split(" ")[-1].replace(";", "")) for rate in data.css("div.shopee-rating-stars__star-wrapper div.shopee-rating-stars__lit")])
        
        product_price = float(data.css_first("div.bPcAVl.IWBsMB span.k9JZlv").text().replace(".", ""))
        product_revenue = convert_to_numeric(data.css_first("div.OwmBnn.eumuJJ").text().split(" ")[-1]) * product_price

        product = {
            'product_name': product_name,
            'product_url': product_url,
            'product_rating': round(product_rating, 2),
            'product_price': round(product_price),
            'product_revenue': round(product_revenue)
        }

        
        products["products"].append(product)
    return products


urls = get_shopee_cat()["urls"].to_list()

for url in urls:
    html = get_html(url)
    print(parse_html(html))
    break

end = datetime.now()
print(end - start)


{
    'products': [
        {
            'product_name': 'Áo khoác cardigan nam mỏng ZF.Club - Dệt kim tổ ong cao cấp CA',
            'product_url': 
'https://shopee.vn/Áo-khoác-cardigan-nam-mỏng-ZF.Club-Dệt-kim-tổ-ong-cao-cấp-CA-i.897161406.23558114113?sp_atk=69f8
7f6a-64de-409a-a946-4640f0f5043d&xptdk=69f87f6a-64de-409a-a946-4640f0f5043d',
            'product_rating': 4.91,
            'product_price': 129000,
            'product_revenue': 85011000
        },
        {
            'product_name': 'Áo khoác cotton có mũ trùm 2 mảnh form rộng kiểu dáng Hàn Quốc unisex dành cho các cặp
đôi mới nhất',
            'product_url': 
'https://shopee.vn/Áo-khoác-cotton-có-mũ-trùm-2-mảnh-form-rộng-kiểu-dáng-Hàn-Quốc-unisex-dành-cho-các-cặp-đôi-mới-n
hất-i.543362608.25555707066?sp_atk=3e4d8b5e-cb45-40b9-808f-5f11419d08e3&xptdk=3e4d8b5e-cb45-40b9-808f-5f11419d08e3'
,
            'product_rating': 2.16,
            'product_price': 190000,
            'product_revenue': 101460000
        },
        {
            'product_name': 'Áo Khoác Gió Nam 5S FASHION Lót Nỉ Dày Dặn,Ấm Áp, Thiết Kế Can Phối Khỏe Khoắn 
(AKG23008)',
            'product_url': 
'https://shopee.vn/Áo-Khoác-Gió-Nam-5S-FASHION-Lót-Nỉ-Dày-Dặn-Ấm-Áp-Thiết-Kế-Can-Phối-Khỏe-Khoắn-(AKG23008)-i.12721
7331.23183247578?sp_atk=05c528d2-00e1-4105-9ade-a5117f426201&xptdk=05c528d2-00e1-4105-9ade-a5117f426201',
            'product_rating': 4.88,
            'product_price': 379000,
            'product_revenue': 236117000
        },
        {
            'product_name': 'Áo Khoác Da Lộn Nam Lót Lông Cừu Cổ Cao Đẹp Cao Cấp ABANDON W1',
            'product_url': 
'https://shopee.vn/Áo-Khoác-Da-Lộn-Nam-Lót-Lông-Cừu-Cổ-Cao-Đẹp-Cao-Cấp-ABANDON-W1-i.327863047.7370925383?sp_atk=cd5
21d24-9c4d-4ea5-9e5e-5c3904abd6b5&xptdk=cd521d24-9c4d-4ea5-9e5e-5c3904abd6b5',
            'product_rating': 4.72,
            'product_price': 259000,
            'product_revenue': 4273500000
        },
        {
            'product_name': 'Áo Khoác Phao Giả Hai Lớp Có Mũ Trùm Dáng Rộng Giữ Ấm Chất Lượng Cao Cho Nam',
            'product_url': 
'https://shopee.vn/Áo-Khoác-Phao-Giả-Hai-Lớp-Có-Mũ-Trùm-Dáng-Rộng-Giữ-Ấm-Chất-Lượng-Cao-Cho-Nam-i.984690466.2388298
8168?sp_atk=fb166440-6048-4db1-9bd5-291b3fb9faef&xptdk=fb166440-6048-4db1-9bd5-291b3fb9faef',
            'product_rating': 4.61,
            'product_price': 509000,
            'product_revenue': 712600000
        },
        {
            'product_name': 'Áo phao NIKE logo thêu ,Áo khoác phao nike béo nam nữ cao cấp,hàng xịn latizia.vn',
            'product_url': 
'https://shopee.vn/Áo-phao-NIKE-logo-thêu-Áo-khoác-phao-nike-béo-nam-nữ-cao-cấp-hàng-xịn-latizia.vn-i.495082115.225
77962692?sp_atk=18c92f42-7d1a-4450-a4b2-1bbff5ff521c&xptdk=18c92f42-7d1a-4450-a4b2-1bbff5ff521c',
            'product_rating': 4.72,
            'product_price': 212000,
            'product_revenue': 445200000
        },
        {
            'product_name': 'Lomon Áo Khoác Len Dày Dặn Thời Trang Mùa Đông Cho Nam Trung Niên',
            'product_url': 
'https://shopee.vn/Lomon-Áo-Khoác-Len-Dày-Dặn-Thời-Trang-Mùa-Đông-Cho-Nam-Trung-Niên-i.302258407.24001375557?sp_atk
=91ec55a5-4324-4e46-8ef1-babb1f2c7a80&xptdk=91ec55a5-4324-4e46-8ef1-babb1f2c7a80',
            'product_rating': 4.93,
            'product_price': 598226,
            'product_revenue': 144172466
        },
        {
            'product_name': 'Áo khoác dạ nam phong cách Hàn Quốc 2 lớp lót dù có nút bấm LADOS-2092 có túi trong, 
ấm áp, sang trọng',
            'product_url': 
'https://shopee.vn/Áo-khoác-dạ-nam-phong-cách-Hàn-Quốc-2-lớp-lót-dù-có-nút-bấm-LADOS-2092-có-túi-trong-ấm-áp-sang-t
rọng-i.35080757.16496050722?sp_atk=e7713dbd-4273-46f7-ac13-07eba72cc32d&xptdk=e7713dbd-4273-46f7-ac13-07eba72cc32d'
,
            'product_rating': 4.88,
            'product_price': 179000,
            'product_revenue': 102030000
        },
        {
            'product_name': 'Áo Khoác Thể Thao Tay Dài Màu Trơn Phong 

0:00:15.893664

---